<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/recruit/fasttext_recruit_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install Flask
!pip install flask_pymongo

In [0]:
!rm -rf fastText
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText && sudo pip install . > /dev/null 2>&1 && sudo python setup.py install > /dev/null 2>&1

Cloning into 'fastText'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 3585 (delta 23), reused 18 (delta 4), pack-reused 3531
Receiving objects: 100% (3585/3585), 8.10 MiB | 26.00 MiB/s, done.
Resolving deltas: 100% (2247/2247), done.


In [0]:
#!wget wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz
#!tar xvzf cooking.stackexchange.tar.gz
# !head cooking.stackexchange.txt

# !rm -rf data
# !mkdir data
# !wget -c http://mattmahoney.net/dc/enwik9.zip -P data
# !unzip data/enwik9.zip -d data     
# !head -c 200 data/fil9 

In [0]:
from flask import Flask
from flask_pymongo import PyMongo
from bs4 import BeautifulSoup
from nltk.tokenize import ToktokTokenizer
from nltk import sent_tokenize, word_tokenize

In [0]:
#__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
# __label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
# __label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
# __label__restaurant Michelin Three Star Restaurant; but if the chef is not there
# sample format needed


app = Flask(__name__)
app.config["MONGO_URI"] = ""
mongo = PyMongo(app)


def cleanMe(html):
    # create a new bs4 object from the html data loaded
    
    # print(html)
    soup = BeautifulSoup(html, "html.parser")
    # remove all javascript and stylesheet code
    for script in soup(["script", "style"]):
        script.extract()
    # get text
    text = soup.get_text(separator=' ')

    # break into lines and remove leading and trailing space on each
    # lines = (line.strip() for line in text.splitlines())
    lines = text.splitlines()
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    # text = '\n'.join(chunk for chunk in chunks if chunk)
    chunks = filter(None, chunks)
    return " ".join(chunks)

ret = mongo.db.naukri_candidates.find({})

all_lines = []


for row in ret:
  education = row["education"]
  workExperiance = row["workExperiance"]
  summary = row['summary']
  key_skills = row["key_skills"]
  projects = row["projects"]
  # extra_data = row["extra_data"]

  summary_str = summary.splitlines()

  education_string = []
  for edu in education:
      education_string.append(edu["type"]  + " " + edu["degree"] + " " + edu["specific"] + " " + edu["university"] + " " + edu["year"])

  work_exp_string = []
  for work in workExperiance:
      work_exp_string.append(work["company_name"]  + " " + work["designation"] + " " + work["date"] + " " + cleanMe(work["desc"]))

  key_skills_str =  " ".join(key_skills.split(",")).replace("Key Skills:","")

  project_str = []
  for project in projects:
      project_str.append(cleanMe(project))

  all_lines.extend(summary_str)
  all_lines.extend(education_string)
  all_lines.extend(work_exp_string)
  all_lines.append(key_skills_str)
  all_lines.extend(project_str)

      # print(row)

print("total lines ", len(all_lines))
            

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


total lines  369931


In [0]:
import nltk
nltk.download('punkt')

toktok = ToktokTokenizer()


open('fastText.train', 'w').close()

with open("fastText.train", 'a') as the_file:
  for line in all_lines:
    tokens = [toktok.tokenize(sent) for sent in sent_tokenize(line)]
    for sent in tokens:
      the_file.write(" ".join(sent) + '\n')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
!head -n 20 fastText.train

Offering 3.5 years of experience in IT , including 1.5 years of experience as Admin &amp; Developer - ServiceNow in Pradyumna
Infotech ( P ) Ltd. ; skilled in System Administration , Automation , Build Engineering , Configuration Management , Monitoring ,
Debugging issues of large scale systems .
Undergone training in configuring ServiceNow tool for various IT Service Management processes like Incident , Problem , Service
Request , Change Management , CMDB and Knowledge Management .
A self motivated person with willingness to learn new skills .
Adept in analyzing information system needs , evaluating end-user
requirements &amp; custom designing solutions .
Expert in serving as the single point of contact between the customer community ( internal and external customers ) and the
software development team through which requirements flow .
Nimble- fingered in monitoring and controlling data security within guidelines to ensure compliance and report on possible
improvements .
UG : B.Com ( 

In [0]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText && mkdir build 
!cd fastText/build && cmake .. && make && make install

fatal: destination path 'fastText' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
-- Configuring done
-- Generating done
-- Build files have been written to: /content/fastText/build
[ 31%] Built target fasttext-static_pic
[ 63%] Built target fasttext-static
[ 68%] Built target fasttext-bin
[100%] Built target fasttext-shared
[ 31%] Built target fasttext-static_pic
[ 63%] Built target fasttext-static
[ 68%] Built target fasttext-bin
[100%] Built target fasttext-shared
Install the project...
-- Install configuration: ""
-- Up-to-date: /usr/local/lib/libfasttext.so
-- Up-to-date: /usr/local/lib/libfasttext.a
-- Up-to-date: /usr/local/lib/libfasttext_pic.a
-- Up-to-date: /usr/local/bin/fasttext
-- Up-to-date: /usr/local/include/fasttext/args.h
-- Up-to-date: /usr/local/include/fasttext/autotune.h
-- Up-to-date: /usr/local/include/fasttext/densematrix.h
-- Up-to-date: /usr/local/include/fasttext/dictionary.h
-- Up-to-date: /usr/local/includ

In [0]:
!fasttext skipgram -input fastText.train -output recruit_fasttext_nosub -dim 300 --minCount 5

Read 22M words
Number of words:  64168
Number of labels: 0
Progress: 100.0% words/sec/thread:    6396 lr:  0.000000 avg.loss:  1.621207 ETA:   0h 0m 0s


In [0]:
# model = fasttext.train_unsupervised('fastText.train',verbose=True, dim=300)

In [0]:
# model.save_model("recruit_fasttext.bin")

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -f recruit_fasttext_nosub.bin /content/drive/My\ Drive/Colab\ Notebooks
!cp -f recruit_fasttext_nosub.vec /content/drive/My\ Drive/Colab\ Notebooks

In [0]:
!cp -f /content/drive/My\ Drive/Colab\ Notebooks/recruit_fasttext.vec .

In [0]:
# print(model)
import fasttext

model = fasttext.load_model("recruit_fasttext.bin")

print("printing similar objects")

print(model.get_nearest_neighbors('excel'))

print(model.get_nearest_neighbors('php'))

print(model.get_nearest_neighbors('delhi'))

print(model.get_nearest_neighbors('seo'))

print(model.get_nearest_neighbors('software developer'))

printing similar objects
[(0.8056473731994629, 'MS-excel'), (0.654314398765564, 'Ms-Excel'), (0.6364229321479797, 'Excel'), (0.6213842034339905, 'excels'), (0.57978755235672, 'MS-Excel'), (0.5385661721229553, 'ppt'), (0.5291121602058411, 'spreadsheet'), (0.5128776431083679, 'VBA'), (0.5106975436210632, 'macros'), (0.5082531571388245, 'spreadsheets')]
[(0.7756870985031128, 'html5'), (0.7689679861068726, 'wpf'), (0.7643250823020935, 'mvvm'), (0.7609511017799377, 'ajax'), (0.7583292722702026, 'jquery'), (0.7557974457740784, 'mq'), (0.7542688250541687, 'sdk'), (0.7525045871734619, 'css3'), (0.7523747682571411, 'mysql'), (0.7513596415519714, 'angularjs')]
[(0.7052754759788513, 'MUmbai'), (0.6987090110778809, 'IIT-Delhi'), (0.689082145690918, 'Delhi'), (0.6754801273345947, 'mumbai'), (0.6582303047180176, 'bahadur'), (0.6568752527236938, 'maharshi'), (0.6520175933837891, 'icfai'), (0.6500081419944763, 'jaipuria'), (0.6487504243850708, 'punjabi'), (0.6473240852355957, 'guru')]
[(0.772792100906

In [0]:
 model.get_analogies("php", "react", "nodejs")

NameError: ignored

In [0]:
!pip install pymagnitude

     |████████████████████████████████| 5.4MB 2.8MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918205 sha256=78c2736e101904cbf8a994464626ea0d8933fe4eb6018a5a420306572f1f6ed3
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude


In [0]:
!python -m pymagnitude.converter -i recruit_fasttext.vec -o recruit_fasttext.magnitude

Loading vectors... (this may take some time)
Found 63983 key(s)
Each vector has 300 dimension(s)
Creating magnitude format...
Writing vectors... (this may take some time)
0% completed
1% completed
2% completed
3% completed
4% completed
5% completed
6% completed
7% completed
8% completed
9% completed
10% completed
11% completed
12% completed
13% completed
14% completed
15% completed
16% completed
17% completed
18% completed
19% completed
20% completed
21% completed
22% completed
23% completed
24% completed
25% completed
26% completed
27% completed
28% completed
29% completed
30% completed
31% completed
32% completed
33% completed
34% completed
35% completed
36% completed
37% completed
38% completed
39% completed
40% completed
41% completed
42% completed
43% completed
44% completed
45% completed
46% completed
47% completed
48% completed
49% completed
50% completed
51% completed
52% completed
53% completed
54% completed
55% completed
56% completed
57% completed
58% completed
59% completed

In [0]:
from pymagnitude import *
vectors = Magnitude("recruit_fasttext.magnitude")

In [0]:
vectors.distance("php", "react")

1.0368235

In [0]:
vectors.distance("php", "laravel")

0.86367863

In [0]:
vectors.distance("php", "html")

0.78127366

In [0]:
vectors.distance("reactjs", "angularjs")

0.76597524

In [0]:
vectors.most_similar("php", topn = 10) 

[('html5', 0.7756865),
 ('wpf', 0.7689677),
 ('mvvm', 0.7643254),
 ('ajax', 0.76095116),
 ('jquery', 0.75832915),
 ('mq', 0.75579786),
 ('sdk', 0.7542695),
 ('css3', 0.7525043),
 ('mysql', 0.7523744),
 ('angularjs', 0.75135964)]

In [0]:
vectors.most_similar("smm", topn = 10) 

[('SMM', 0.7502389560987464),
 ('SM', 0.7483717703957413),
 ('SMO', 0.5689362523627867),
 ('SEO/SEM', 0.5539700271959749),
 ('SEO', 0.5234724338375396),
 ('SMI', 0.5186737307009426),
 ('SM9', 0.5104172533923532),
 ('SEM', 0.5103022570687503),
 ('CSMM', 0.5002657639189045),
 ('Marketing/SEM', 0.4977302609256824)]

In [0]:
vectors.most_similar("consultant", topn = 10) 

[('consultant/client', 0.84090364),
 ('client/consultant', 0.8016337),
 ('consultants/', 0.7588308),
 ('Consultant', 0.75776714),
 ('consultants', 0.7199213),
 ('Consultant(', 0.71790504),
 ('Consultant/', 0.7045239),
 ('Consultant-ERP', 0.67827797),
 ('Architect/Consultant', 0.67308253),
 ('Consultant-SAP', 0.67237467)]

In [0]:
vectors.most_similar_cosmul("infosys", topn = 10) 

[('Infosys', 0.9373685),
 ('Infosystem', 0.8866384),
 ('Infosystems', 0.86907554),
 ('Infosoft', 0.84773844),
 ('INFOSYS', 0.83583313),
 ('Infosol', 0.8247505),
 ('infosystem', 0.817755),
 ('infosystems', 0.807857),
 ('WIPRO', 0.80352575),
 ('Infostretch', 0.7980992)]

In [0]:
vectors.most_similar("mca", topn = 50) 

[('MCA', 0.9168030113845977),
 ('YMCA', 0.6532520722240681),
 ('BCA', 0.6135635671880891),
 ('Bio-Chemistry/Bio-Technology', 0.5848759111725454),
 ('Computers', 0.5783263504821089),
 ('M.Tech', 0.5625866122195022),
 ('Electronics/Telecommunication', 0.5547224362843186),
 ('Bio-Chemistry', 0.5490236222235825),
 ('M.Com.', 0.5439897946051826),
 ('B.El.Ed', 0.5362615401939946),
 ('LLB', 0.531547636758143),
 ('Production/Industrial', 0.5309321105845726),
 ('M.Com', 0.5296752590383729),
 ('B.N', 0.5246129448612419),
 ('B.Pharma', 0.5240113452281342),
 ('M.C.A', 0.5191827885479195),
 ('HR/Industrial', 0.5176870969145939),
 ('MCAD', 0.5170587691422704),
 ('Arts&Humanities', 0.5156220360600348),
 ('B.Sc.', 0.5145124367967958),
 ('M.Pharma', 0.5085329181150453),
 ('3CA', 0.5056585208638642),
 ('B.Ed', 0.5054883267850967),
 ('B.SC', 0.5029315069602465),
 ('B.I.T', 0.5025359706191423),
 ('23ACA', 0.5023474850642082),
 ('MS/M.Sc(', 0.5015636205404442),
 ('LLM', 0.5007666371872305),
 ('B.P.U.T', 0.

In [0]:
vectors.most_similar(["technical","writing"], topn = 50) 

[('technical/non-technical', 0.7444383),
 ('technical/functional', 0.736218),
 ('technical/', 0.7193762),
 ('functional/technical', 0.71644646),
 ('project/technical', 0.71449804),
 ('writings', 0.7117934),
 ('writing/', 0.70930135),
 ('nontechnical', 0.7023362),
 ('technicality', 0.69422936),
 ('technicalities', 0.6856699),
 ('educational/technical', 0.67780983),
 ('handwriting', 0.67652273),
 ('technical/commercial', 0.66872233),
 ('*Writing', 0.6616802),
 ('business/technical', 0.66013163),
 ('customizing/technical', 0.6555693),
 ('copywriting', 0.6412802),
 ('rewriting', 0.6273825),
 ('non-technical', 0.6215545),
 ('proof-reading', 0.6120782),
 ('Non-technical', 0.61027217),
 ('writting', 0.60148203),
 ('operational/functional', 0.594646),
 ('copy-editing', 0.5912042),
 ('proofreading', 0.5899374),
 ('Technical/Functional', 0.5889574),
 ('Functional/Technical', 0.580129),
 ('technically', 0.5794834),
 ('Proof-reading', 0.57773095),
 ('business/functional', 0.5727279),
 ('Write/revi